#  create NOAA DSDP taxa list for import
## 1-96 taxa

create taxa list to import in database

In [14]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR, RAW_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_ID, PBDB_TAXA_NAME
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb

In [15]:

date='2024-04-03'


base_dir = CLEAN_DATA_DIR

initial_input_file = RAW_DATA_DIR/'PI_processed_files'/'NOAA'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
input_file = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_pbdb_{date}.csv"
crosswalk_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_crosswalk_{date}.csv"
taxa_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_list_{date}.csv"

genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_pbdb_{date}.csv'
higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_pbdb_{date}.csv'  

taxa_pbdb_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_list_pbdb_{date}.csv'

# old_genus_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'genus_pbdb_{prev_date}.csv'
# old_higher_taxa_pbdb_file = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'higher_taxa_pbdb_{prev_date}.csv'  



## cleanup taxa list


In [52]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)
# (7763, 39)

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [53]:
PI_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments',
       'pbdb_taxon_id_old', 'pbdb_taxon_name', 'pbdb_taxon_rank',
       'Corrections to pbdb_taxon_rank', 'pbdb_taxon_id', 'pbdb_taxon_name.1',
       'pbdb_taxon_rank.1', 'Corrections to pbdb_taxon', 'family_taxon_id',
       'family_taxon_name', 'superfamily_taxon_id', 'superfamily_taxon_name',
       'order_taxon_id', 'order_taxon_name', 'class_taxon_id',
       'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name',
       'kingdom_taxon_id', 'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [54]:
if 'pbdb_taxon_name.1' in PI_df:
    print('update cols')

    PI_df = PI_df.rename(columns={ 
        # 'pbdb_taxon_id': 'pbdb_taxon_id_old',
        'pbdb_taxon_name': 'pbdb_taxon_name_old',
        'pbdb_taxon_rank': 'pbdb_taxon_rank_old',
        # 'pbdb_taxon_id.1': 'pbdb_taxon_id',
        'pbdb_taxon_name.1': 'pbdb_taxon_name',
        'pbdb_taxon_rank.1': 'pbdb_taxon_rank'
    })

update cols


2. QA corrections to taxa id

In [55]:
cols = [
    'Corrections to pbdb_taxon_rank',
    'Corrections to pbdb_taxon',
    'pbdb_taxon_id_old',
    'pbdb_taxon_id'
]
tmp = PI_df[PI_df['Corrections to pbdb_taxon_rank'].notna() | 
            PI_df['Corrections to pbdb_taxon'].notna()].copy()

tmp[cols].drop_duplicates()

,Corrections to pbdb_taxon_rank,Corrections to pbdb_taxon,pbdb_taxon_id_old,pbdb_taxon_id
295,NaN,1059,383509,1059
299,genus; id 1064,1064,168467,1064
438,"genus, id 1124",NaN,70260,1124
1169,genus taxon_no=2092,2092,153914,2092
1411,genus taxon_no=2542,NaN,137071,2542
1640,NaN,441243,374615,441243
1752,genus; ID 71247,NaN,9512,71247
1989,genus; ID 82145,NaN,14728,82145
2144,478034,478034,NaN,478034
2195,441540,441540,83895,441540


In [56]:
PI_df.shape
# 7763

(7763, 39)

In [57]:
PI_df.to_csv(input_file, index=False)


# More QA

In [72]:
df = pd.read_csv(input_file, dtype=str)


In [73]:
df[df['pbdb_taxon_id'].isna()].shape

(17, 39)

In [74]:
df[df['pbdb_taxon_id'].isna() & 
   df['Any taxon above genus'].isna() &
  df['genus name'].isna()]

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name


2. taxa with missing pbdb taxon id that needs to be added

In [75]:
df[df['Any taxon above genus'] == 'Globigerinidae indet.']


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
4431,planktic_forams,Forams planktonic indent.,Globigerinidae indet.,NaN,Joc manually filled in the taxon nos for this,NaN,Globigerinidae indet.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [76]:
# pbdb.fix_taxa_for_row_id(df, 4431, 82191)

3. update taxa with missing pbdb taxon name and rank

In [77]:
tmp = df[df['pbdb_taxon_name'].isna() & df['pbdb_taxon_id'].notna()][['pbdb_taxon_id']]
tmp.shape

(61, 1)

In [78]:
for index, row in tmp.iterrows():
    if row['pbdb_taxon_id']== '454654':
        continue
    pbdb.fix_taxa_for_row_id(df, index, row['pbdb_taxon_id'])


440987
478034
443687
443687
443806
443806
443806
443806
443806
276935
276935
276935
264667
477767
477767
409479
409481
409479
409479
409479
410976
410976
424279
424279
424279
424279
424279
424279
87836
453595
453595
270678
256803
24409
2825
2825
249963
308943
252756
454352
252828
478723
345975
338622
277351
418507
250494
483433
277167
259514
159808
159808
428
625
159912
650
85877
85877
85877
454938


3b. manually add taxon name and rank for taxa where the taxa lineage has changed 

In [87]:
df[df['pbdb_taxon_name'].isna() & (df['pbdb_taxon_id']=='454654')]
df.at[6835, 'pbdb_taxon_name']='Pterocyrtidium'
df.at[6835, 'pbdb_taxon_rank']='genus'


4. update pbdb data for taxa with outdated info

In [79]:
df[df['pbdb_taxon_id']=='443687']

,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus_taxon_id,genus_taxon_name,corrected
2950,dinoflagellates,Ascostomocystis potane,NaN,NaN,"in PBDB, so it will be classified correctly wi...",NaN,NaN,NaN,Ascostomocystis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443687,Ascostomocystis,True
2951,dinoflagellates,Ascostomocystis sp.,NaN,NaN,NaN,NaN,NaN,NaN,Ascostomocystis,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443687,Ascostomocystis,True


In [80]:
pbdb.fix_taxa_for_row_id(df, 2950, 443687, True)
pbdb.fix_taxa_for_row_id(df, 2951, 443687, True)

443687
443687


In [81]:
# del df['corrected']

In [88]:
df.to_csv(input_file, index=False)


## create crosswalk file

In [93]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [94]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [95]:
crosswalk_df = nt.create_taxa_crosswalk_df(df)
log_df(crosswalk_df)
# 7763
# 7760
# 7759

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'verbatim_name', 'name comment field', 'Comment', 'Notes (change to Internal only notes?)', 'comments']
initial df:  (7763, 17)
remove nontaxa df:  (7763, 17)
drop duplicates df:  (7762, 17)
(7762, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Abyssamina,,,,incisa,,,,Abyssamina incisa,benthic_forams,Abyssamina incisa,NaN,,,
1,,,Adercotryma,,,,glomeratum,,,,Adercotryma glomeratum,benthic_forams,Adercotryma glomeratum,NaN,,,
2,,,Adercotryma,,,,sp.,,,,Adercotryma sp.,benthic_forams,Adercotryma sp.,NaN,,,
3,,,Alabamina,,,,decorata,,,,Alabamina decorata,benthic_forams,Alabamina decorata,NaN,,,
4,,,Alabamina,,,,haitiensis,,,,Alabamina haitiensis,benthic_forams,Alabamina haitiensis,NaN,,,


In [96]:
PI_df.to_csv(crosswalk_file, index=False)

## create taxa file

In [101]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [102]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [103]:
taxa_df = nt.create_taxa_list_df(df)
log_df(taxa_df)
# 7763
# 7760
# 7658

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id', 'family_taxon_name', 'order_taxon_id', 'order_taxon_name', 'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name']
initial df:  (7763, 25)
remove nontaxa df:  (7763, 25)
drop duplicates df:  (7656, 25)
(7656, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Abyssamina,,,,incisa,,,,...,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,,,Adercotryma,,,,glomeratum,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,,,Adercotryma,,,,sp.,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
3,,,Alabamina,,,,decorata,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria
4,,,Alabamina,,,,haitiensis,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria


In [104]:
taxa_df.to_csv(taxa_file, index=False)